# 📦 Microsoft Fabric – Capacity Migration Inventory

This notebook **auto-discovers** every Fabric capacity, workspace, and item in
your tenant to produce a **migration-ready inventory** for cross-region
capacity moves (e.g., West US → East US 2).

| Metric | Source |
|--------|--------|
| Capacity Name, Region, SKU, State | `GET /v1/capacities` |
| Workspace Name, Capacity assignment | `GET /v1/admin/workspaces` |
| Item Name, Type, Last Updated | `GET /v1/admin/items` |
| Movable vs Non-Movable classification | [MS Learn – Capacity reassignment restrictions](https://learn.microsoft.com/en-us/fabric/admin/portal-workspace-capacity-reassignment) |
| Migration Complexity | Calculated rule-based |
| Suggested Wave | Auto-assigned by complexity |

```mermaid
flowchart LR
    A["🔐 Auth (auto)"] --> B["📡 FabricRestClient"]
    B --> C["🏢 List Capacities"]
    B --> D["📂 List Workspaces (Admin)"]
    B --> E["📦 List Items (Admin)"]
    C --> F["📊 Inventory DataFrame"]
    D --> F
    E --> F
    F --> G["✅ Movable / ❌ Non-Movable"]
    G --> H["📈 Migration Complexity"]
    H --> I["💾 Export CSV"]
```

## Prerequisites
- This notebook runs **inside Microsoft Fabric** (not externally)
- **Fabric Administrator** role (required for Admin APIs)
- **No external dependencies** – uses only Fabric built-in libraries:
  - `sempy.fabric.FabricRestClient` – REST API calls with automatic auth
  - `pandas` – pre-installed in Fabric runtimes

## Key Microsoft Learn References
| Topic | Link |
|-------|------|
| Capacity reassignment restrictions | https://learn.microsoft.com/en-us/fabric/admin/portal-workspace-capacity-reassignment |
| Multi-Geo support | https://learn.microsoft.com/en-us/fabric/admin/service-admin-premium-multi-geo |
| Move Power BI tenant to different region | https://learn.microsoft.com/en-us/power-bi/admin/service-admin-region-move |
| Find Fabric home region | https://learn.microsoft.com/en-us/fabric/admin/find-fabric-home-region |
| Admin Items API | https://learn.microsoft.com/en-us/rest/api/fabric/admin/items/list-items |
| Admin Workspaces API | https://learn.microsoft.com/en-us/rest/api/fabric/admin/workspaces/list-workspaces |
| Capacities API | https://learn.microsoft.com/en-us/rest/api/fabric/core/capacities/list-capacities |

## 0️⃣ Verify Fabric Environment & Retry Helper

In [ ]:
# ── Verify Fabric Environment ─────────────────────────────────────────
# Zero external dependencies – only Fabric built-in libs + Python stdlib

import sempy.fabric as fabric
import pandas as pd
import json
from datetime import datetime, timedelta, timezone
import time
import random

# Initialize Fabric REST client (handles auth automatically via notebook identity)
rest_client = fabric.FabricRestClient()

print("✅ Fabric environment verified (zero external dependencies)")
print(f"   pandas version: {pd.__version__}")
print(f"   REST client:    sempy.fabric.FabricRestClient (auto-auth)")


# ── Resilient API Helper with Retry & Exponential Backoff ────────────

def fabric_api_get(client, path, max_retries=5, base_delay=1.0,
                   max_delay=60.0):
    """
    GET request via FabricRestClient with automatic retry + exponential
    backoff for HTTP 429 (rate-limit) and transient 5xx errors.
    Honors the Retry-After header when present.

    Uses only Fabric built-in libs – no 'requests' import needed.
    """
    retryable = {429, 500, 502, 503, 504}

    for attempt in range(max_retries + 1):
        try:
            resp = client.get(path)

            if resp.status_code not in retryable:
                return resp

            # Determine wait time
            if resp.status_code == 429:
                retry_after = resp.headers.get("Retry-After")
                wait = float(retry_after) if retry_after else base_delay * (2 ** attempt)
            else:
                wait = base_delay * (2 ** attempt)

            wait = min(wait * (0.75 + random.random() * 0.5), max_delay)  # jitter + cap

            if attempt < max_retries:
                print(f"   ⏳ HTTP {resp.status_code} – retrying in {wait:.1f}s "
                      f"(attempt {attempt + 1}/{max_retries})")
                time.sleep(wait)
            else:
                resp.raise_for_status()

        except (ConnectionError, OSError):
            if attempt < max_retries:
                wait = min(base_delay * (2 ** attempt), max_delay)
                print(f"   ⏳ Connection error – retrying in {wait:.1f}s "
                      f"(attempt {attempt + 1}/{max_retries})")
                time.sleep(wait)
            else:
                raise

    return resp


print("✅ Retry helper loaded (max 5 retries, exponential backoff with jitter)")

## 1️⃣ Configuration

No workspace-specific config needed – this notebook scans the **entire tenant**
using Fabric Admin APIs.  Optionally set a target region filter.

In [ ]:
# ── Configuration ────────────────────────────────────────────────────

# Filter: only inventorise capacities in this source region (set to None for all)
SOURCE_REGION_FILTER = None   # e.g. "West US", "East US 2", or None for all

# Target region (used for labelling in migration plan)
TARGET_REGION = "East US 2"

# Auth is handled automatically by FabricRestClient (initialized above)
# No manual tokens, headers, or API base URLs needed.

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_colwidth', 50)

print(f"📦 Fabric Capacity Migration Inventory")
print(f"   Source Region Filter: {SOURCE_REGION_FILTER or 'ALL'}")
print(f"   Target Region:       {TARGET_REGION}")

## 2️⃣ Discover All Capacities

Uses `GET /v1/capacities` to list every capacity the caller can access.
Returns capacity ID, display name, SKU, region, and state.

In [ ]:
# ── List Capacities ───────────────────────────────────────────────

def list_all_capacities(client):
    """List all Fabric capacities via FabricRestClient with pagination."""
    path = "/v1/capacities"
    capacities = []
    while path:
        resp = fabric_api_get(client, path)
        resp.raise_for_status()
        data = resp.json()
        capacities.extend(data.get("value", []))
        cont = data.get("continuationToken")
        path = f"/v1/capacities?continuationToken={cont}" if cont else None
    return capacities


raw_capacities = list_all_capacities(rest_client)

df_capacities = pd.DataFrame([
    {
        "capacity_id":   c["id"],
        "capacity_name": c["displayName"],
        "sku":           c["sku"],
        "region":        c["region"],
        "state":         c["state"],
    }
    for c in raw_capacities
])

# Apply region filter if specified
if SOURCE_REGION_FILTER:
    df_capacities = df_capacities[
        df_capacities["region"].str.lower() == SOURCE_REGION_FILTER.lower()
    ].copy()

print(f"🏢 Found {len(df_capacities)} capacity/ies")
print()
print(df_capacities[["capacity_name", "sku", "region", "state"]].to_string(index=False))

## 3️⃣ Discover All Workspaces

Uses the **Admin API** `GET /v1/admin/workspaces` to list every workspace,
then joins to capacities to get region info.

> **Rate limit:** Admin API allows max 200 requests / hour.

In [ ]:
# ── List All Workspaces (Admin API) ──────────────────────────────

def list_admin_workspaces(client, capacity_id=None, state="Active"):
    """List workspaces via Admin API with pagination (uses FabricRestClient)."""
    path = f"/v1/admin/workspaces?state={state}"
    if capacity_id:
        path += f"&capacityId={capacity_id}"

    workspaces = []
    while path:
        resp = fabric_api_get(client, path)
        resp.raise_for_status()
        data = resp.json()
        workspaces.extend(data.get("workspaces", []))
        cont = data.get("continuationToken")
        if cont:
            base = path.split("&continuationToken")[0].split("?continuationToken")[0]
            sep = "&" if "?" in base else "?"
            path = f"{base}{sep}continuationToken={cont}"
        else:
            path = None
        time.sleep(0.5)  # be gentle on rate limits
    return workspaces


# Fetch workspaces scoped to our capacity list
all_workspaces = []
target_capacity_ids = set(df_capacities["capacity_id"])

for _, cap_row in df_capacities.iterrows():
    print(f"   📂 Scanning capacity: {cap_row['capacity_name']} ({cap_row['region']}) ...")
    ws_list = list_admin_workspaces(rest_client, capacity_id=cap_row["capacity_id"])
    for ws in ws_list:
        all_workspaces.append({
            "workspace_id":   ws["id"],
            "workspace_name": ws["name"],
            "workspace_type": ws.get("type", "Workspace"),
            "workspace_state": ws.get("state", "Active"),
            "capacity_id":    ws.get("capacityId", ""),
        })
    time.sleep(1.0)

df_workspaces = pd.DataFrame(all_workspaces)

# Join capacity details
if not df_workspaces.empty:
    df_workspaces = df_workspaces.merge(
        df_capacities[["capacity_id", "capacity_name", "sku", "region"]],
        on="capacity_id", how="left"
    )

print(f"\n✅ Found {len(df_workspaces)} workspace(s) across {len(df_capacities)} capacity/ies")

## 4️⃣ Discover All Items per Workspace

Uses the **Admin Items API** `GET /v1/admin/items` to enumerate every item,
then classifies each as **Movable** or **Non-Movable** per the
[capacity reassignment restrictions](https://learn.microsoft.com/en-us/fabric/admin/portal-workspace-capacity-reassignment).

### Cross-Region Movable Item Types
| Movable ✅ | Non-Movable ❌ (must recreate) |
|-----------|-------------------------------|
| Report | Lakehouse |
| Semantic model (small) | Warehouse |
| Dashboard | Notebook |
| Dataflow Gen1 | Eventhouse |
| Paginated Report | KQLDatabase |
| Datamart | DataPipeline |
| Scorecard | SparkJobDefinition |
| | Eventstream |
| | MLExperiment / MLModel |
| | Environment |
| | SQLEndpoint |
| | All other Fabric-native items |

In [ ]:
# ── List All Items (Admin API) & Classify Movability ────────────────

# Per MS Learn: cross-region movable types
MOVABLE_TYPES = {
    "Report", "SemanticModel", "Dashboard",
    "DataflowGen1", "Dataflow",
    "PaginatedReport", "Datamart", "Scorecard",
}


def list_admin_items(client, workspace_id=None, capacity_id=None):
    """List items via Admin API with pagination (uses FabricRestClient)."""
    path = "/v1/admin/items"
    params_parts = []
    if workspace_id:
        params_parts.append(f"workspaceId={workspace_id}")
    if capacity_id:
        params_parts.append(f"capacityId={capacity_id}")
    if params_parts:
        path += "?" + "&".join(params_parts)

    items = []
    while path:
        resp = fabric_api_get(client, path)
        resp.raise_for_status()
        data = resp.json()
        items.extend(data.get("itemEntities", []))
        cont = data.get("continuationToken")
        if cont:
            base = path.split("&continuationToken")[0].split("?continuationToken")[0]
            sep = "&" if "?" in base else "?"
            path = f"{base}{sep}continuationToken={cont}"
        else:
            path = None
        time.sleep(0.5)
    return items


# Fetch items for each capacity in scope
all_items = []
for _, cap_row in df_capacities.iterrows():
    print(f"   📦 Scanning items in capacity: {cap_row['capacity_name']} ...")
    raw_items = list_admin_items(rest_client, capacity_id=cap_row["capacity_id"])
    for item in raw_items:
        itype = item.get("type", "Unknown")
        all_items.append({
            "item_id":        item["id"],
            "item_name":      item.get("name", ""),
            "item_type":      itype,
            "workspace_id":   item.get("workspaceId", ""),
            "capacity_id":    item.get("capacityId", ""),
            "last_updated":   item.get("lastUpdatedDate", ""),
            "is_movable":     itype in MOVABLE_TYPES,
            "movability":    "✅ Movable" if itype in MOVABLE_TYPES else "❌ Non-Movable",
        })
    print(f"      {len(raw_items)} item(s) found")
    time.sleep(1.0)

df_items = pd.DataFrame(all_items)

# Join workspace + capacity details
if not df_items.empty and not df_workspaces.empty:
    df_items = df_items.merge(
        df_workspaces[["workspace_id", "workspace_name", "capacity_name", "sku", "region"]],
        on="workspace_id", how="left", suffixes=("", "_ws")
    )

print(f"\n✅ Total items discovered: {len(df_items)}")
if not df_items.empty:
    movable     = df_items[df_items["is_movable"] == True]
    non_movable = df_items[df_items["is_movable"] == False]
    print(f"   ✅ Movable:     {len(movable)}")
    print(f"   ❌ Non-Movable: {len(non_movable)}")
    print(f"\n📊 Item Type Distribution:")
    print(df_items.groupby(["item_type", "movability"]).size()
          .reset_index(name="count")
          .sort_values(["movability", "count"], ascending=[True, False])
          .to_string(index=False))

## 5️⃣ Calculate Migration Complexity per Workspace

Assigns a **Low / Medium / High** migration complexity to each workspace
based on the presence of non-movable items and their types.

| Complexity | Criteria |
|------------|----------|
| **Low** | Only movable items (Reports, Semantic models, Dashboards, etc.) |
| **Medium** | Has Lakehouses or Warehouses (data copy required) |
| **High** | Has Notebooks, Pipelines, Eventhouses, or other complex Fabric items |

In [ ]:
# ── Migration Complexity & Wave Assignment ────────────────────────

HIGH_COMPLEXITY_TYPES = {
    "Notebook", "DataPipeline", "Eventhouse", "KQLDatabase",
    "SparkJobDefinition", "Eventstream", "MLExperiment", "MLModel",
    "Environment", "KQLQueryset", "KQLDashboard", "GraphQLApi",
    "MirroredDatabase", "MirroredWarehouse", "Reflex",
    "DataflowGen2", "CopyJob", "ApacheAirflowJob",
}

MEDIUM_COMPLEXITY_TYPES = {
    "Lakehouse", "Warehouse", "SQLEndpoint", "SQLDatabase",
}


def assess_workspace_complexity(ws_items_df):
    """Determine migration complexity for a workspace based on its items."""
    types_present = set(ws_items_df["item_type"])
    non_movable_count = ws_items_df[~ws_items_df["is_movable"]].shape[0]

    if types_present & HIGH_COMPLEXITY_TYPES:
        complexity = "High"
    elif types_present & MEDIUM_COMPLEXITY_TYPES:
        complexity = "Medium"
    elif non_movable_count > 0:
        complexity = "Medium"
    else:
        complexity = "Low"

    return complexity


def assign_wave(complexity):
    """Auto-assign migration wave based on complexity."""
    return {"Low": 1, "Medium": 2, "High": 3}.get(complexity, 3)


# Build workspace-level summary
ws_summaries = []

if not df_items.empty:
    for ws_id, ws_group in df_items.groupby("workspace_id"):
        ws_name = ws_group["workspace_name"].iloc[0] if "workspace_name" in ws_group.columns else "Unknown"
        cap_name = ws_group["capacity_name_ws"].iloc[0] if "capacity_name_ws" in ws_group.columns else (
            ws_group["capacity_name"].iloc[0] if "capacity_name" in ws_group.columns else "Unknown"
        )
        cap_sku = ws_group["sku"].iloc[0] if "sku" in ws_group.columns else "Unknown"
        cap_region = ws_group["region"].iloc[0] if "region" in ws_group.columns else "Unknown"

        total_items = len(ws_group)
        movable_count = ws_group["is_movable"].sum()
        non_movable_count = total_items - movable_count

        types_present = ws_group["item_type"].value_counts().to_dict()
        non_movable_types = ws_group[~ws_group["is_movable"]]["item_type"].unique().tolist()

        complexity = assess_workspace_complexity(ws_group)
        wave = assign_wave(complexity)

        ws_summaries.append({
            "workspace_id":        ws_id,
            "workspace_name":      ws_name,
            "capacity_name":       cap_name,
            "sku":                 cap_sku,
            "source_region":       cap_region,
            "target_region":       TARGET_REGION,
            "total_items":         total_items,
            "movable_items":       int(movable_count),
            "non_movable_items":   int(non_movable_count),
            "item_types_summary":  str(types_present),
            "non_movable_types":   ", ".join(non_movable_types) if non_movable_types else "",
            "migration_complexity": complexity,
            "suggested_wave":      wave,
            "business_owner":      "",      # ← fill manually
            "data_size_estimate":  "",      # ← fill manually
            "external_deps":      "",      # ← fill manually (e.g. Databricks, Gateways)
            "notes":              "",
        })

df_ws_summary = pd.DataFrame(ws_summaries)

if not df_ws_summary.empty:
    df_ws_summary = df_ws_summary.sort_values(["suggested_wave", "migration_complexity"])

print(f"✅ Workspace migration assessment complete: {len(df_ws_summary)} workspace(s)")
print(f"   Wave 1 (Low):    {len(df_ws_summary[df_ws_summary['suggested_wave'] == 1])}")
print(f"   Wave 2 (Medium): {len(df_ws_summary[df_ws_summary['suggested_wave'] == 2])}")
print(f"   Wave 3 (High):   {len(df_ws_summary[df_ws_summary['suggested_wave'] == 3])}")

## 6️⃣ Migration Readiness Report

In [ ]:
# ── Migration Readiness Report ────────────────────────────────────

if not df_ws_summary.empty:
    print("📦 FABRIC CAPACITY MIGRATION – READINESS REPORT")
    print("=" * 130)
    print(f"   Source Region:   {SOURCE_REGION_FILTER or 'ALL'}")
    print(f"   Target Region:   {TARGET_REGION}")
    print(f"   Capacities:      {len(df_capacities)}")
    print(f"   Workspaces:      {len(df_ws_summary)}")
    print(f"   Total Items:     {df_ws_summary['total_items'].sum():,}")
    print(f"   Movable Items:   {df_ws_summary['movable_items'].sum():,}")
    print(f"   Non-Movable:     {df_ws_summary['non_movable_items'].sum():,}")
    print("=" * 130)

    # Per-wave breakdown
    for wave in [1, 2, 3]:
        wave_ws = df_ws_summary[df_ws_summary["suggested_wave"] == wave]
        if wave_ws.empty:
            continue
        label = {1: "Low (Movable-only)", 2: "Medium (Data copy)", 3: "High (Recreate artifacts)"}[wave]
        print(f"\n🟢 WAVE {wave}: {label} – {len(wave_ws)} workspace(s)")
        print("-" * 130)
        for _, row in wave_ws.iterrows():
            print(f"   📂 {row['workspace_name']}")
            print(f"      Capacity: {row['capacity_name']} ({row['sku']}) | "
                  f"Region: {row['source_region']} → {row['target_region']}")
            print(f"      Items: {row['total_items']} total | "
                  f"✅ {row['movable_items']} movable | "
                  f"❌ {row['non_movable_items']} non-movable")
            if row['non_movable_types']:
                print(f"      Non-movable types: {row['non_movable_types']}")
            print()

    # Constraints reminder
    print("\n⚠️  KEY CONSTRAINTS (from MS Learn)")
    print("=" * 100)
    print("   1. Workspace reassignment CANCELS all running jobs")
    print("   2. Non-movable items must be REMOVED before cross-region reassignment")
    print("   3. Non-movable items must be RECREATED in the target region")
    print("   4. Dataflow Gen2 staging lakehouses/warehouses block migration")
    print("      (delete all Dataflow Gen2 items first, then delete staging items)")
    print("   5. Large-storage format semantic models cannot be moved between regions")
    print("   6. Private Link networking must be temporarily disabled during migration")
    print("   7. After migration, it can take up to 1 hour before new Fabric items")
    print("      can be created in the workspace")
    print()
    print("   📚 References:")
    print("   • https://learn.microsoft.com/en-us/fabric/admin/portal-workspace-capacity-reassignment")
    print("   • https://learn.microsoft.com/en-us/fabric/admin/service-admin-premium-multi-geo")
    print("   • https://learn.microsoft.com/en-us/power-bi/admin/service-admin-region-move")
else:
    print("⚠️  No workspace data to report.")

## 7️⃣ Phased Migration Plan Summary

Auto-generated phased plan based on the inventory.

In [ ]:
# ── Phased Migration Plan ─────────────────────────────────────────

print("📋 PHASED MIGRATION PLAN")
print("=" * 120)

phases = [
    ("Phase 0", "Discovery & Inventory", [
        "✅ Run this notebook to auto-discover all capacities, workspaces, items",
        "Export inventory CSV and share with business owners",
        "Identify business owners for each workspace",
        "Document external dependencies (Databricks, gateways, VNets)",
        "Confirm data sizes for lakehouses and warehouses",
        "Agree on migration windows with stakeholders",
    ]),
    ("Phase 1", "Target Capacity Setup in " + TARGET_REGION, [
        f"Provision new Fabric capacity/ies in {TARGET_REGION}",
        "Match SKU sizing to source capacities",
        "Configure networking (VNet, private endpoints if needed)",
        "Set up gateways in the target region",
        "Validate admin & RBAC permissions on new capacities",
    ]),
    ("Phase 2", "Wave 1 – Low Complexity (Movable-only workspaces)", [
        "Reassign workspaces with ONLY movable items to target capacity",
        "Movable types: Report, Semantic model, Dashboard, Dataflow Gen1, Paginated Report, Datamart, Scorecard",
        "Validate reports and semantic models post-migration",
        "Update data source connections if needed",
    ]),
    ("Phase 3", "Wave 2 – Medium Complexity (Data copy required)", [
        f"Create new Lakehouses/Warehouses in {TARGET_REGION} workspaces",
        "Copy data using Fabric Copy Job, AzCopy, or notebooks",
        "Remove non-movable items from source workspace",
        "Reassign workspace to target capacity",
        "Recreate Lakehouses/Warehouses and validate data",
        "Reconnect semantic models and reports to new endpoints",
    ]),
    ("Phase 4", "Wave 3 – High Complexity (Recreate artifacts)", [
        "Backup notebook code, pipeline definitions, eventhouse configs",
        "Use Git integration to version-control definitions",
        "Remove all non-movable items from source workspace",
        "Reassign workspace to target capacity",
        "Recreate Notebooks, Pipelines, Eventhouses, Spark jobs",
        "Copy lakehouse data and re-point pipeline sources",
        "Run end-to-end validation of data pipelines",
    ]),
    ("Phase 5", "Validation, Cutover & Decommission", [
        "Run parallel execution for critical workloads",
        "Business sign-off on data accuracy and report functionality",
        "Update IP allowlists, DNS, and Azure Private Endpoints",
        "Decommission source capacities in old region",
        "Archive migration documentation",
    ]),
]

for phase_id, phase_name, steps in phases:
    print(f"\n📌 {phase_id}: {phase_name}")
    print("-" * 90)
    for i, step in enumerate(steps, 1):
        print(f"   {i}. {step}")

## 8️⃣ Export Inventory to CSV

Exports three CSV files:
1. **Capacities** – all capacities with region and SKU
2. **Workspace Summary** – migration complexity, wave, movable/non-movable counts
3. **Item Detail** – every item with movability classification

In [ ]:
# ── Export Inventory to CSV ────────────────────────────────────────

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# Capacities
if not df_capacities.empty:
    f1 = f"migration_capacities_{timestamp}.csv"
    df_capacities.to_csv(f1, index=False)
    print(f"✅ Capacities exported: {f1} ({len(df_capacities)} rows)")

# Workspace summary
if not df_ws_summary.empty:
    f2 = f"migration_workspace_summary_{timestamp}.csv"
    df_ws_summary.to_csv(f2, index=False)
    print(f"✅ Workspace summary exported: {f2} ({len(df_ws_summary)} rows)")

# Item detail
if not df_items.empty:
    f3 = f"migration_item_detail_{timestamp}.csv"
    export_cols = [c for c in [
        "item_id", "item_name", "item_type", "movability",
        "workspace_id", "workspace_name", "capacity_name",
        "sku", "region", "last_updated"
    ] if c in df_items.columns]
    df_items[export_cols].to_csv(f3, index=False)
    print(f"✅ Item detail exported: {f3} ({len(df_items)} rows)")

print("\n💾 Use these CSV files to populate the Excel Migration Tracker template.")
print("   See: fabric-region-migration/generate_migration_tracker_excel.py")

---

## 📝 Notes

### Zero External Dependencies

This notebook uses **only Fabric built-in libraries** – no `pip install` required:

| Library | Purpose | Pre-installed |
|---------|---------|---------------|
| `sempy.fabric.FabricRestClient` | REST API calls with automatic auth | ✅ Yes (semantic-link) |
| `pandas` | DataFrames, analysis, CSV export | ✅ Yes |
| `json`, `time`, `random`, `datetime` | Python standard library | ✅ Yes |

### Fabric REST API Endpoints Used

| Endpoint | Purpose | Rate Limit |
|----------|---------|------------|
| `GET /v1/capacities` | List all capacities | Standard |
| `GET /v1/admin/workspaces` | List all workspaces (tenant-wide) | 200 req/hr |
| `GET /v1/admin/items` | List all items (tenant-wide) | 200 req/hr |

### Cross-Region Movable Items (MS Learn)

Only these types can move between regions via workspace reassignment:
- Report, Semantic model (small storage), Dashboard
- Dataflow Gen1, Paginated Report, Datamart, Scorecard

**All other Fabric-native items** (Lakehouse, Warehouse, Notebook, Pipeline,
Eventhouse, etc.) must be **deleted from the source workspace**, the workspace
reassigned, then the items **recreated in the target region**.

### Permissions Required
- **Fabric Administrator** or **Service Principal** with `Tenant.Read.All` scope
- For capacity listing: `Capacity.Read.All`

### Key Microsoft Learn References
- [Capacity reassignment restrictions](https://learn.microsoft.com/en-us/fabric/admin/portal-workspace-capacity-reassignment)
- [Multi-Geo support for Fabric](https://learn.microsoft.com/en-us/fabric/admin/service-admin-premium-multi-geo)
- [Move Power BI tenant to different region](https://learn.microsoft.com/en-us/power-bi/admin/service-admin-region-move)
- [Find your Fabric home region](https://learn.microsoft.com/en-us/fabric/admin/find-fabric-home-region)
- [Admin Items API](https://learn.microsoft.com/en-us/rest/api/fabric/admin/items/list-items)
- [Admin Workspaces API](https://learn.microsoft.com/en-us/rest/api/fabric/admin/workspaces/list-workspaces)
- [Capacities API](https://learn.microsoft.com/en-us/rest/api/fabric/core/capacities/list-capacities)